In [1]:
import gdown

In [2]:
# url of the data in gdrive
url = "https://drive.google.com/file/d/1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR/view?usp=sharing"

In [9]:
# Getting the unique ID from the url
file_id = url.split('/')[-2]
file_id

'1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR'

In [10]:
# Common for the all and passing the file id at the end
prefix = 'https://drive.google.com/uc?/export=download&id='

In [11]:
gdown.download(prefix+file_id, "Data.zip")

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR
From (redirected): https://drive.google.com/uc?/export=download&id=1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR&confirm=t&uuid=41a49d98-d0e5-4489-90ca-d1572743302c
To: d:\Kidney-Disease-Classification\notebooks\Data.zip
100%|██████████| 1.63G/1.63G [00:52<00:00, 31.2MB/s]


'Data.zip'

In [3]:
## Predicting the image from saved model
import os
os.chdir('../')

In [8]:
import torch
from PIL import Image
from torchvision import transforms

In [4]:

model = torch.load("artifacts\models\model.pth",
                           map_location=torch.device('cpu')).to('cpu')

In [33]:
image = Image.open(r'artifacts\data_ingestion\Data\Normal\Normal- (5074).jpg')

In [34]:
data_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])

transformed_img = data_transform(image).unsqueeze(0)

In [37]:
pred = torch.softmax(model(transformed_img), dim=1)
pred 

tensor([[0.0255, 0.7688, 0.0520, 0.1536]], grad_fn=<SoftmaxBackward0>)

In [24]:
import numpy as np


In [28]:
class_names = ['Cyst', 'Normal', 'Stone', 'Tumor']

In [40]:
def predict(image_path):
      
    image = Image.open(image_path)
    
    # Transform the target image and add a batch dimension
    img = data_transform(image).unsqueeze(0)
    
    # Put model into evaluation mode and turn on inference mode
    model.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(model(img), dim=1)
    
    # Create a prediction label and prediction probability dictionary for each prediction class.
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    Predictions = dict(sorted(pred_labels_and_probs.items(), key=lambda item: item[1], reverse=True))
    return Predictions

In [41]:
predict(r"artifacts\data_ingestion\Data\Normal\Normal- (5074).jpg")

{'Normal': 0.7688122987747192,
 'Tumor': 0.153639554977417,
 'Stone': 0.05202086269855499,
 'Cyst': 0.025527214631438255}